In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import cv2
import urllib.request
import numpy as np
import os
from google.colab.patches import cv2_imshow

# cifar100 = keras.datasets.cifar100
# (x_train, y_train), (x_test, y_test) = cifar100.load_data()
# x_train = x_train.astype('float32') / 255.0
# x_test = x_test.astype('float32') / 255.0

cifar10 = keras.datasets.cifar10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

model = keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(32, 32, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
    layers.Flatten(),
    layers.Dense(100, activation='softmax')
])


model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

history = model.fit(x_train, y_train, epochs=5, validation_data=(x_test, y_test))

yolo_dir = 'yolo'
yolo_weights_url = 'https://pjreddie.com/media/files/yolov3.weights'
yolo_cfg_url = 'https://raw.githubusercontent.com/AlexeyAB/darknet/master/cfg/yolov3.cfg'
yolo_names_url = 'https://raw.githubusercontent.com/pjreddie/darknet/master/data/coco.names'
os.makedirs(yolo_dir, exist_ok=True)
urllib.request.urlretrieve(yolo_weights_url, os.path.join(yolo_dir, 'yolov3.weights'))
urllib.request.urlretrieve(yolo_cfg_url, os.path.join(yolo_dir, 'yolov3.cfg'))
urllib.request.urlretrieve(yolo_names_url, os.path.join(yolo_dir, 'coco.names'))
with open(os.path.join(yolo_dir, 'coco.names'), 'r') as f:
    yolo_labels = [line.strip() for line in f.readlines()]

net = cv2.dnn.readNetFromDarknet(os.path.join(yolo_dir, 'yolov3.cfg'), os.path.join(yolo_dir, 'yolov3.weights'))

colors = np.random.uniform(0, 255, size=(len(yolo_labels), 3))

urllib.request.urlretrieve('https://plus.unsplash.com/premium_photo-1663040394761-0c609ee63f16?ixlib=rb-4.0.3&ixid=MnwxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8&auto=format&fit=crop&w=1740&q=80', 'test_image.jpg')
image = cv2.imread('test_image.jpg')
blob = cv2.dnn.blobFromImage(image, 1/255, (416, 416), swapRB=True, crop=False)
net.setInput(blob)
outputs = net.forward(net.getUnconnectedOutLayersNames())
conf_threshold = 0.8
nms_threshold = 0.4
class_ids = []
confidences = []
object_boxes=[]
face_boxes=[]
boxes = []
for output in outputs:
  for detection in output:
    scores = detection[5:]
    class_id = np.argmax(scores)
    confidence = scores[class_id]
    if confidence > conf_threshold and class_id != 81 :
      center_x = int(detection[0] * image.shape[1])
      center_y = int(detection[1] * image.shape[0])
      width = int(detection[2] * image.shape[1])
      height = int(detection[3] * image.shape[0])
      x = center_x - width // 2
      y = center_y - height // 2
      class_ids.append(class_id)
      confidences.append(float(confidence))
      boxes.append([x, y, width, height])
      object_box = [x, y, width, height]
      object_boxes.append(object_box)
indices = cv2.dnn.NMSBoxes(boxes, confidences, conf_threshold, nms_threshold)
for i in indices:
  box = boxes[i]
  x, y, w, h = box
  color = [int(c) for c in colors[class_ids[i]]]
  cv2.rectangle(image, (x, y), (x + w, y + h), color, 2)
  text = f"{yolo_labels[class_ids[i]]}: {confidences[i]:.2f}"
  cv2.putText(image, text, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)


cv2_imshow(image)